## **Four Square APIs**

This tutorial provides a brief description of how getting started with Foursquare API. You will also find a good tutorial here:
https://www.kaggle.com/code/kristoft/tutorial-foursquare-api-search


**Getting Started**

Foursquare API provides a range of tools for developers to incorporate the up-to-date location data to enhance their projects.

To start using Foursquare, first, let’s create a new account at https://developer.foursquare.com/ . The default account is called Sandbox Account. Sandbox Account provides you a limited set of tools, for instance, you‘ll be able to make only 950 Regular Calls / Day, and get 1 Photo and 1 Tip per Venue. Not too much, but it should be enough for a “toy” dataset (sure, you may also upgrade your account for future projects). Foursquare provides a simple UI to manage your account and to check your daily stats (like how many calls did you make). Once the account is created, you will be assigned credentials: the Client ID and the Client Secret.

Ok, all set up and we can jump into coding. First and foremost, let’s import some necessary libraries: pandas.io.json(to tranforming json file into a pandas dataframe library), folium (plotting library), geopy.geocoders (module to convert an address into latitude and longitude values), and requests (library to handle requests).

In [1]:
!pip install geopy
from pandas import json_normalize
import folium
import pandas as pd
from geopy.geocoders import Nominatim
import requests


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


Next, lets define our Foursquare credentials and version into python code:

In [2]:
CLIENT_ID = "AX1O0GXCKC2YOCKBYWQRXPXQFLZBAZE5PKHGOPA5QRMJQWKI" # your Foursquare ID
CLIENT_SECRET = "XH34Z0ZQXNJSOWAL5EP4000O0UBYMJVI1UEFJTPKO0SZF24G" # your Foursquare Secret
VERSION = "20180604"
LIMIT = 100

Now, let’s assume you just moved to a new place which called the VUB and you want to explore your neighbourhood. So, let’s define a query to search the top 100 venues that are within 2500 meters from the VUB

In [3]:
# Select first building (which in our case is Penn):
neighborhood_name = 'VUB'

neighborhood_latitude = 50.8222
neighborhood_longitude = 4.3969

# limit of number of venues returned by Foursquare API
LIMIT = 100
radius = 2500

And now we define the corresponding URL and send the GET Request to Foursquare API and examine the results.

In [4]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    neighborhood_latitude,
    neighborhood_longitude,
    radius,
    LIMIT)
results = requests.get(url).json()
print(results)

{'meta': {'code': 200, 'requestId': '67cf15e3f13a8b56c9c25bdd'}, 'response': {'queryRefinements': {'target': {'type': 'path', 'url': '/venue/explore', 'params': {'ll': '50.822200,4.396900', 'radius': '2500'}}, 'refinements': [{'query': 'Food'}, {'query': 'Nightlife'}, {'query': 'Coffee'}, {'query': 'Shops'}, {'query': 'Arts'}, {'query': 'Outdoors'}]}, 'suggestedFilters': {'header': 'Tap to show:', 'filters': [{'name': 'Open now', 'key': 'openNow'}]}, 'headerLocation': 'Brussels', 'headerFullLocation': 'Brussels', 'headerLocationGranularity': 'city', 'totalResults': 179, 'suggestedBounds': {'ne': {'lat': 50.84470002250003, 'lng': 4.432450073905012}, 'sw': {'lat': 50.79969997749998, 'lng': 4.361349926094987}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4be871c288ed2d7f15e1cb1d', 'name': "Luigi's Café", 'cont

Next, get relevant part of JSON and transform it into a pandas dataframe:

In [5]:
# assign relevant part of JSON to venues
venues = results['response']['groups'][0]['items']

# tranform venues into a pandas dataframe
dataframe = json_normalize(venues)
dataframe.head()


,tips,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.contact.phone,venue.contact.formattedPhone,venue.location.address,venue.location.lat,...,venue.storeId,venue.menu.type,venue.menu.label,venue.menu.anchor,venue.menu.url,venue.menu.mobileUrl,venue.menu.externalUrl,venue.location.crossStreet,venue.venueRatingBlacklisted,venue.contact.instagram
0,"[{'id': '4fcb5115e4b0e6dca7c4d538', 'createdAt...",e-0-4be871c288ed2d7f15e1cb1d-0,0,"[{'summary': 'This spot is popular', 'type': '...",4be871c288ed2d7f15e1cb1d,Luigi's Café,+3226469296,+32 2 646 92 96,Generaal Jacqueslaan 253 Boulevard Général Jac...,50.821815,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[{'id': '508d03f1e4b0645d6766ce57', 'createdAt...",e-0-4f87fe32e4b005979cdc0156-1,0,"[{'summary': 'This spot is popular', 'type': '...",4f87fe32e4b005979cdc0156,Kings of Comedy Club,+3226499930,+32 2 649 99 30,Boondaalsesteenweg 489 Chaussée de Boondael,50.815248,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"[{'id': '4b8d0ae070c603bb796293b4', 'createdAt...",e-0-4b649474f964a5204abf2ae3-2,0,"[{'summary': 'This spot is popular', 'type': '...",4b649474f964a5204abf2ae3,Le Gauguin,+3226463972,+32 2 646 39 72,Boondaalsesteenweg 420 Chaussée de Boondael,50.817488,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[{'id': '59e87d0ca4b51b0e62d4c5fe', 'createdAt...",e-0-524abf5a11d22ca199294388-3,0,"[{'summary': 'This spot is popular', 'type': '...",524abf5a11d22ca199294388,Les Tartes de Françoise,+3226726272,+32 2 672 62 72,Square Jean-Baptiste De Greef 2,50.820510,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"[{'id': '4faa4ad2e4b0328fbea2d135', 'createdAt...",e-0-4b94038ef964a520566134e3-4,0,"[{'summary': 'This spot is popular', 'type': '...",4b94038ef964a520566134e3,Villa Singha,+3226756734,+32 2 675 67 34,22 Rue des Trois Ponts / Driebruggenstraat 22,50.820634,...,NaN,Menu,Menu,View Menu,http://www.singha.be/html/menu.htm,http://www.singha.be/html/menu.htm,http://www.singha.be/html/menu.htm,NaN,NaN,NaN



Define information of interest and filter the dataframe

In [6]:
# filter columns
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']

    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =dataframe.loc[:, filtered_columns]# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head(10)

,name,categories,lat,lng
0,Luigi's Café,Pub,50.821815,4.388259
1,Kings of Comedy Club,Comedy Club,50.815248,4.389900
2,Le Gauguin,Bar,50.817488,4.388301
3,Les Tartes de Françoise,Pie Shop,50.820510,4.408903
4,Villa Singha,Thai Restaurant,50.820634,4.407810
5,Fresh Med,Supermarket,50.832297,4.403824
6,Peter & Sabine,Butcher,50.826114,4.412382
7,Pizza Liloo,Pizzeria,50.832825,4.386597
8,Les Tartes de Françoise,Pie Shop,50.820312,4.376851
9,Mexigo,Taco Restaurant,50.811424,4.389857


Finally, lets visualize the venues around Penn. For this purpose I use Folium library (Folium is completely free). Generating the map is straigtforward in Folium. We only create a Folium Map object and display it. So, I generate a map centred around the Penn campus. Bellow is the python code:

In [7]:
venues_map = folium.Map(location=[neighborhood_latitude, neighborhood_longitude], zoom_start=15)

folium.features.CircleMarker(
    [neighborhood_latitude, neighborhood_longitude],
    radius=10,
    color='red',
    popup='UPENN',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)


# add all venues as blue circle markers
for lat, lng, label in zip(nearby_venues.lat, nearby_venues.lng, nearby_venues.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

venues_map